In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [ ]:

import os
import pandas as pd 
import numpy as np



In [ ]:
# !git clone https://github.com/jason9693/Soongsil-BERT.git

In [ ]:
%cd Soongsil-BERT/finetune

[Errno 2] No such file or directory: 'Soongsil-BERT/finetune'
/content/drive/MyDrive


In [ ]:
!sudo apt-get update
!sudo apt-get install build-essential
!sudo apt-get install python3-dev


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.8ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remo

In [ ]:
!pip install tokenizers==0.5.2
!pip install --upgrade transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tokenizers-0.5.2.tar.gz (64 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install soynlp emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import numpy as np
import glob

from fastprogress.fastprogress import master_bar, progress_bar
from sklearn import metrics as sklearn_metrics

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    
    AdamW,
    get_linear_schedule_with_warmup
)

In [ ]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
NVIDIA A100-SXM4-40GB


In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100-SXM4-40GB


In [ ]:
model_name = "Haaaaeun/kcbert_hatespeech"
# print(f"  Model path = {args.model_name_or_path}")
# print(f"  output_dir = {args.output_dir}")

label_list = ["none", "hate"]

config = AutoConfig.from_pretrained(
    model_name
    )
tokenizer = AutoTokenizer.from_pretrained(
      model_name
    )
model = AutoModelForSequenceClassification.from_pretrained(
    model_name
    )

In [ ]:
args = {
  "task": "hate-speech",
  "data_dir": "data",
  "ckpt_dir": "ckpt",
  "train_file": "train.tsv",
  "dev_file": "dev.csv",
  "test_file": "val.tsv",
  "max_seq_len": 128,
  "num_train_epochs": 5,
  "weight_decay": 0.0,
  "gradient_accumulation_steps": 1,
  "adam_epsilon": 1e-8,
  "warmup_proportion": 0,
  "max_steps": -1,
  "max_grad_norm": 1.0,
  "no_cuda": False,
  "model_name_or_path": f"{model_name}",
  "output_dir": f"{model_name}-hate-speech-ckpt",
  "seed": 42,
  "train_batch_size": 32,
  "eval_batch_size": 64,
  "logging_steps": 300,
  "save_steps": 300,
  "learning_rate": 5e-5,
  "evaluate_test_during_training": True,
  "eval_all_checkpoints": True,
  "save_optimizer": False,
  "do_lower_case": False,
  "do_train": True,
  "do_eval": True,
  "model_type": "roberta"}

In [ ]:
# GPU or CPU
args["device"] = "cuda" if torch.cuda.is_available() and not args["no_cuda"] else "cpu"
#GPU 사용
device = torch.device("cuda:0")

In [ ]:

model.to(args["device"])
print("Finish")


Finish


In [ ]:
%cd '/content/drive/MyDrive/dataset/dataset/Soongsil-BERT/finetune/data/hate-speech'


import os
import pandas as pd 
import numpy as np

# 데이터 준비
test_file = "dev.csv"
test = pd.read_csv(test_file)
test = test.dropna(axis=0)
test

/content/drive/.shortcut-targets-by-id/19AmDlGex-hSZJyofrcVWB6uu3yQV-PSB/dataset/dataset/Soongsil-BERT/finetune/data/hate-speech


,문장,혐오 여부
0,정말 재밌다 연기도 좋고 디카프리오 짱,0
1,심쿵심쿵 미치네요이수혁땜에 잠 못자겠어요ㅠ자꾸 아른거림ㅠ,0
2,하지만 이니후빨러들은 이런거 관심 하나도 없음,1
3,@착한아이임당 A4 용지 덮고?,1
4,"진짜 평점 믿으면 안된 다는 걸 현실로 보여주는 영화입니다. 네티즌,전문가 하나같이...",0
...,...,...
189990,원작을 읽을 때 이런 건 절대 영상화하기 힘들다고 생각했는데 벤휘쇼의 연기와 더불어...,0
189991,케석대 어깨 올라간거봐라 ㅋㅋ,1
189992,@김짜꾸 day and night\n\nround the clock\n\nwitho...,1
189993,로버트다우니주니어를 좋아해서 봤는데너무재밌게 봤던영화생각없이 볼때 딱좋음,0


In [ ]:
import pandas as pd

# '혐오 여부' 열의 값을 1에서 0으로, 0에서 1로 바꾸기
# test['혐오 여부'] = test['혐오 여부'].apply(lambda x: 0 if x == 1 else 1)

# 변경된 데이터프레임을 'dev.csv' 파일로 저장 (덮어쓰기)
# test.to_csv('dev.csv', index=False)

# 결과 출력
print(test)


                                                       문장  혐오 여부
0                                   정말 재밌다 연기도 좋고 디카프리오 짱      0
1                         심쿵심쿵 미치네요이수혁땜에 잠 못자겠어요ㅠ자꾸 아른거림ㅠ      0
2                               하지만 이니후빨러들은 이런거 관심 하나도 없음      1
3                                       @착한아이임당 A4 용지 덮고?      1
4       진짜 평점 믿으면 안된 다는 걸 현실로 보여주는 영화입니다. 네티즌,전문가 하나같이...      0
...                                                   ...    ...
189990  원작을 읽을 때 이런 건 절대 영상화하기 힘들다고 생각했는데 벤휘쇼의 연기와 더불어...      0
189991                                   케석대 어깨 올라간거봐라 ㅋㅋ      1
189992  @김짜꾸 day and night\n\nround the clock\n\nwitho...      1
189993           로버트다우니주니어를 좋아해서 봤는데너무재밌게 봤던영화생각없이 볼때 딱좋음      0
189994                         @익명_146173 개지랄병 병신좌좀새끼ㅋㅋㅋㅋ      1

[189995 rows x 2 columns]


In [ ]:
args

{'task': 'hate-speech',
 'data_dir': 'data',
 'ckpt_dir': 'ckpt',
 'train_file': 'train.tsv',
 'dev_file': 'dev.csv',
 'test_file': 'val.tsv',
 'max_seq_len': 128,
 'num_train_epochs': 5,
 'weight_decay': 0.0,
 'gradient_accumulation_steps': 1,
 'adam_epsilon': 1e-08,
 'warmup_proportion': 0,
 'max_steps': -1,
 'max_grad_norm': 1.0,
 'no_cuda': False,
 'model_name_or_path': 'Haaaaeun/kcbert_hatespeech',
 'output_dir': 'Haaaaeun/kcbert_hatespeech-hate-speech-ckpt',
 'seed': 42,
 'train_batch_size': 32,
 'eval_batch_size': 64,
 'logging_steps': 300,
 'save_steps': 300,
 'learning_rate': 5e-05,
 'evaluate_test_during_training': True,
 'eval_all_checkpoints': True,
 'save_optimizer': False,
 'do_lower_case': False,
 'do_train': True,
 'do_eval': True,
 'model_type': 'roberta',
 'device': 'cuda'}

In [ ]:
import csv

def load_data(tokenizer):
  file_to_read = args["dev_file"]
  input_path = os.path.join(args["data_dir"], args["task"], file_to_read)
  print(f"LOOKING AT {input_path}")
    
  texts = []
  labels = []
    
  with open(input_path, "r", encoding="utf-8") as f:
    csv_reader = csv.reader(f)
    next(csv_reader)  # 첫 번째 줄은 헤더이므로 건너뜁니다.
        
    for i, line in enumerate(csv_reader):
      text_a = line[0]
          
      if text_a == "":
        continue
            
      if i % 100000 == 0:
        print(f"[{i}] {line}")
            
      texts.append({
        "text_a": text_a,
        "text_b": None,
      })
      labels.append(int(line[1]))
  
  # print(texts)
  
  # Convert text data to feature
  batch_encoding = tokenizer(
      [text["text_a"] for text in texts],
      max_length=args["max_seq_len"],
      truncation=True,
      padding=True,
      return_tensors='pt',
      return_token_type_ids=True
  )

  features = []
  for i in range(len(texts)):
    input = {k: batch_encoding[k][i] for k in  batch_encoding}
    if "token_type_ids" not in input:
      input["token_type_ids"] = [0] * len(input["input_ids"])
    features.append(input)

  for i, feature in enumerate(features[:5]):
    print("*** Example ***")
    print("input_ids: {}".format(" ".join([str(x) for x in feature["input_ids"]])))
    print("attention_mask: {}".format(" ".join([str(x) for x in feature["attention_mask"]])))
    print("token_type_ids: {}".format(" ".join([str(x) for x in feature["token_type_ids"]])))
    print("label: {}".format(labels[i]))

  # Convert feature to dataset
  all_input_ids = batch_encoding["input_ids"]
  all_attention_mask = batch_encoding["attention_mask"]
  all_token_type_ids = batch_encoding["token_type_ids"]
  all_labels = torch.tensor(labels, dtype=torch.long)

  dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
  return dataset

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Base Model (370M)

tokenizer = AutoTokenizer.from_pretrained("jason9693/soongsil-bert-base")

In [ ]:
tokenizer.tokenize("안녕 에..? 지금은 테스트중")

['ìķĪëħķ', 'ĠìĹĲ', '..?', 'Ġì§Ģê¸ĪìĿĢ', 'ĠíħĮìĬ¤íĬ¸', 'ì¤ĳ']

In [ ]:
batch_encoding = tokenizer(
      [text for text in test], padding=True, return_tensors='pt')

In [ ]:
batch_encoding

{'input_ids': tensor([[   0,  514,  409,    2,    1],
        [   0, 4065,  474, 9159,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1]])}

In [ ]:
test[:3]

,문장,혐오 여부
0,정말 재밌다 연기도 좋고 디카프리오 짱,1
1,심쿵심쿵 미치네요이수혁땜에 잠 못자겠어요ㅠ자꾸 아른거림ㅠ,1
2,하지만 이니후빨러들은 이런거 관심 하나도 없음,0


In [ ]:
test

,문장,혐오 여부
0,정말 재밌다 연기도 좋고 디카프리오 짱,1
1,심쿵심쿵 미치네요이수혁땜에 잠 못자겠어요ㅠ자꾸 아른거림ㅠ,1
2,하지만 이니후빨러들은 이런거 관심 하나도 없음,0
3,@착한아이임당 A4 용지 덮고?,0
4,"진짜 평점 믿으면 안된 다는 걸 현실로 보여주는 영화입니다. 네티즌,전문가 하나같이...",1
...,...,...
189990,원작을 읽을 때 이런 건 절대 영상화하기 힘들다고 생각했는데 벤휘쇼의 연기와 더불어...,1
189991,케석대 어깨 올라간거봐라 ㅋㅋ,0
189992,@김짜꾸 day and night\n\nround the clock\n\nwitho...,0
189993,로버트다우니주니어를 좋아해서 봤는데너무재밌게 봤던영화생각없이 볼때 딱좋음,1


In [ ]:
%cd '/content/drive/MyDrive/dataset/dataset/Soongsil-BERT/finetune'
test_dataset = load_data(tokenizer)

/content/drive/.shortcut-targets-by-id/19AmDlGex-hSZJyofrcVWB6uu3yQV-PSB/dataset/dataset/Soongsil-BERT/finetune
LOOKING AT data/hate-speech/dev.csv
[0] ['정말 재밌다 연기도 좋고 디카프리오 짱', '1']
[100000] ['요즘 청와대 하는거 보면 진짜 암걸리겠다..문재인 대통령되고 나서 모든게 엉망이다', '0']
*** Example ***
input_ids: tensor(0) tensor(2082) tensor(2945) tensor(275) tensor(802) tensor(1174) tensor(3888) tensor(1909) tensor(1118) tensor(830) tensor(7653) tensor(2479) tensor(2) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor(1) tensor

In [ ]:
def evaluate(_model, _eval_dataset, mode, _global_step=None):
  results = {}
  eval_sampler = SequentialSampler(_eval_dataset)
  eval_dataloader = DataLoader(_eval_dataset, sampler=eval_sampler, batch_size=args["eval_batch_size"])

  # Eval!
  if _global_step != None:
    print(f"***** Running evaluation on {mode} dataset ({_global_step} step) *****")
  else:
    print(f"***** Running evaluation on {mode} dataset *****")
  print(f"  Num examples = {len(_eval_dataset)}")
  print(f"  Eval Batch size = {args['eval_batch_size']}")
  eval_loss = 0.0
  nb_eval_steps = 0
  preds = None
  out_label_ids = None

  # Dataloader [for]
  for batch in progress_bar(eval_dataloader):
  
    batch = tuple(t.to(args["device"]) for t in batch)
    # if nb_eval_steps % 100000 == 0:
      # print(nb_eval_steps, end=" ")
      # print("batch:")
      # print(batch)

    with torch.no_grad():
      inputs = {
          "input_ids": batch[0],
          "attention_mask": batch[1],
          "labels": batch[3]
      }
      if args["model_type"] not in ["distilkobert", "xlm-roberta"]:
        inputs["token_type_ids"] = batch[2]  # Distilkobert, XLM-Roberta don't use segment_ids
      # inputs["attention_type"] = 'original_full'

      outputs = _model(**inputs)
      # if nb_eval_steps % 100000 == 0:
        # print("outputs:")
        # print(outputs)

      tmp_eval_loss, logits = outputs[:2]
      # if nb_eval_steps % 100000 == 0:
        # print("tmp_eval_loss, logits:") 
        # print(tmp_eval_loss, logits)

      eval_loss += tmp_eval_loss.mean().item()
      
    nb_eval_steps += 1

    if preds is None:
      preds = logits.detach().cpu().numpy()
      out_label_ids = inputs["labels"].detach().cpu().numpy()
    else:
      preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
      out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)
  # End Dataloader [for]

  eval_loss = eval_loss / nb_eval_steps
  preds = np.argmax(preds, axis=1)
  result = {
      "precision": sklearn_metrics.precision_score(out_label_ids, preds, average="macro"),
      "recall": sklearn_metrics.recall_score(out_label_ids, preds, average="macro"),
      "f1": sklearn_metrics.f1_score(out_label_ids, preds, average="macro")
  }
  results.update(result)

  output_dir = os.path.join(args["output_dir"], "evaluate")
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  output_eval_file = os.path.join(output_dir, f"{mode}-{_global_step}.txt" if _global_step else f"{mode}.txt")
  with open(output_eval_file, "w") as f_w:
    print(f"***** Eval results on {mode} dataset *****")
    for key in sorted(results.keys()):
      print(f"  {key} = {str(results[key])}")
      f_w.write(f"  {key} = {str(results[key])}\n")

  return results

In [ ]:
results = evaluate(model, test_dataset, mode="test")
print("Evaluation results on test dataset:")
for key, value in results.items():
    print(f"{key} = {value}")

***** Running evaluation on test dataset *****
  Num examples = 189995
  Eval Batch size = 64


Attention type 'block_sparse' is not possible if sequence_length: 128 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


***** Eval results on test dataset *****
  f1 = 0.3457055066564746
  precision = 0.6664615806930637
  recall = 0.5003266676962037
Evaluation results on test dataset:
precision = 0.6664615806930637
recall = 0.5003266676962037
f1 = 0.3457055066564746


In [ ]:
# # Train!
# print("***** Running training *****")
# print(f"  Traning model_ = {args.model_name_or_path}")
# print(f"  Num examples = {len(train_dataset)}")
# print(f"  Num Epochs = {args.num_train_epochs}")
# print(f"  Total train batch size = {args.train_batch_size}")
# print(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
# print(f"  Total optimization steps = {t_total}")
# print(f"  Logging steps = {args.logging_steps}")
# print(f"  Save steps = { args.save_steps}")

global_step = 1
epochs_trained = 0.0
steps_trained_in_current_epoch = 0

if os.path.exists(args.output_dir):
  try:

    print("  Find latest checkpoint")
    ckpts_suffix = [int(c.split("-")[-1]) for c in os.listdir(args.output_dir) if c.startswith("checkpoint")] 
    global_step = max(ckpts_suffix)
    ckpt_path = os.path.join(args.output_dir, f"checkpoint-{global_step}" )
    
    model = AutoModelForSequenceClassification.from_pretrained(ckpt_path)
    model.to(args.device)

    epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
    steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)
    print("  Continuing training from checkpoint, will skip to saved global_step")
    print(f"  Continuing training from epoch {epochs_trained}")
    print(f"  Continuing training from global step {global_step}", )
    print(f"  Will skip the first {steps_trained_in_current_epoch} steps in the first epoch")
  except ValueError:
    print("  Starting fine-tuning.")

tr_loss = 0.0

model.zero_grad()
mb = master_bar(range(int(args.num_train_epochs)))
for epoch in mb:
  epoch_iterator = progress_bar(train_dataloader, parent=mb)
  # One epoch train
  for step, batch in enumerate(epoch_iterator):
    # Skip past any already trained steps if resuming training
    if steps_trained_in_current_epoch > 0:
      steps_trained_in_current_epoch -= 1
      continue
      
    model.train()
    batch = tuple(t.to(args.device) for t in batch)
    inputs = {
        "input_ids":batch[0],
        "attention_mask": batch[1],
        "labels": batch[3]
    }

    if args.model_type not in ["distilkobert", "xlm-roberta"]:
      inputs["token_type_ids"] = batch[2]  # Distilkobert, XLM-Roberta don't use segment_ids
    outputs = model(**inputs)

    loss = outputs[0]

    if args.gradient_accumulation_steps > 1:
      loss = loss / args.gradient_accumulation_steps

    loss.backward()
    tr_loss += loss.item()
    # Accumulation 
    if (step + 1) % args.gradient_accumulation_steps == 0 or (
        len(train_dataloader) <= args.gradient_accumulation_steps 
        and (step + 1) == len(train_dataloader)
    ):
      torch.nn.utils.clip_grad_norm(model.parameters(), args.max_grad_norm)

      optimizer.step()
      scheduler.step()
      model.zero_grad()
      global_step += 1

      evaluate(model, dev_dataset, mode="dev", _global_step=global_step)
      

      # Save model checkpoint
      if args.save_steps > 0 and global_step % args.save_steps == 0:
        output_dir = os.path.join(args.output_dir, f"checkpoint-{global_step}")
        if not os.path.exists(output_dir):
          os.makedirs(output_dir)
        model_to_save = model.module if hasattr(model, "module") else model
        model_to_save.save_pretrained(output_dir)

        torch.save(args, os.path.join(output_dir, "training_args.bin"))
        print(f" Saving model checkpoint to {output_dir}")

        if args.save_optimizer:
          torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
          torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
      # End Save model [if]
    # End Accumulation [if]
    
    if args.max_steps > 0 and global_step > args.max_steps:
      break
  # End One epoch train [for]
  mb.write("Epoch {} done".format(epoch + 1))

  if args.max_steps > 0 and global_step > args.max_steps:
    break 
tr_loss = tr_loss / global_step

AttributeError: ignored

In [ ]:
epoch

In [ ]:
print(f"global_step = {global_step}, average loss = {tr_loss}")

In [ ]:
checkpoints = list(
      os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + "pytorch_model.bin", recursive=True))
)

In [ ]:
results = {}
for checkpoint in checkpoints:
  _gloabl_step = checkpoint.split("-")[-1]
  eval_model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
  eval_model.to(args.device)
  eval_model.eval()
  result = evaluate(eval_model, test_dataset, mode="test", _global_step=_gloabl_step)
  result = dict((k +f"_{_gloabl_step}", v) for k, v in result.items())
  results.update(result)

In [ ]:
for key in sorted(results.keys()):
  print(f"{key} = {results[key]}")